# Load Packages and Settings

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from gurobipy import *
import random
from itertools import chain

In [ ]:
def figsize(scale):
    fig_width_pt = 503.295     # Get this from LaTeX using \the\textwidth
    inches_per_pt = 1.0/72.27   # Convert pt to inch
    golden_mean = (np.sqrt(5.0)-1.0)/2.0    # Aesthetic ratio (you could change this)
    fig_width = fig_width_pt*inches_per_pt*scale # width in inches
    fig_height = fig_width*golden_mean  # height in inches
    fig_size = [fig_width,fig_height]
    return fig_size

publication_with_latex = {  # setup matplotlib to use latex for output
    "pgf.texsystem": "pdflatex", # change this if using xetex or lautex
    "text.usetex": True, # use LaTeX to write all text
    "font.family": "serif",
    "axes.labelsize": 8,  # LaTeX default is 10pt font.
    "font.size": 8,
    "legend.fontsize": 8,  # Make the legend/label fonts a little smaller
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    }
matplotlib.rcParams.update(publication_with_latex)
matplotlib.rcParams['savefig.dpi'] = 125
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath,amssymb,amsfonts}"]

# Plot Average Optimal Precision Curve and Results of Algorithms

In [ ]:
n = 50
P = {}
eps_list = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.985]
opt_val_list = np.zeros(len(eps_list))
iter_input = 1;
# Assuming p_uv's are generated for 10 different graph realizations from Preferential Attachment model
for iter_input_ind in range(10): 
    iter_input =  iter_input_ind+1
    str = "../build/temp_model/P_matrix_RW_ratio_{0}.txt".format(iter_input_ind) #location of ith graph matrix
    print(str)
    data_t = open(str,'r')
    data_t = data_t.readlines()
    temp_l=data_t[2].split()
    n = int(temp_l[0]) 
    for lin in data_t[3:]:
        temp_l=lin.split()
        P[int(temp_l[0]), int(temp_l[1])] = float(temp_l[2])

    for ind,eps in enumerate(eps_list):
        m = Model("Optimization in Node Age Problem")
        
        #Add variables and objective function
        y = {}
        nC2 = n*(n-1)/2
        s = m.addVar(lb=0.0, ub=1/(eps*nC2), obj=0, vtype=GRB.CONTINUOUS, name="s")        
        for i in range(n):
            for j in chain(range(i),range(i+1,n)): #TRICK: to combine range in Python 3
                y[i,j] = m.addVar(lb=0.0, obj=-P[i,j], vtype = GRB.CONTINUOUS, name="y[%s,%s]"%(i,j))
        m.update()
        
        #Antisymmetry constraints
        var = []
        for i in range(n):
            for j in chain(range(i),range(i+1,n)):
                m.addConstr(y[i,j], GRB.LESS_EQUAL, s)
                m.addConstr(y[i,j]+y[j,i], GRB.LESS_EQUAL, s)
                var.append(y[i,j])
        # Constraint that \sum y_{u,v} = 1
        coef = [1 for j in range(n*(n-1))]
        m.addConstr(LinExpr(coef,var), "=", 1)

        #Transitivity constraints
        for i in range(n):
            for j in chain(range(i),range(i+1,n)):
                k_1 = min(i,j)
                k_2 = max(i,j)
                for k in chain(range(k_1), range(k_1+1,k_2), range(k_2+1,n)):
                    m.addConstr(y[i,j]+y[j,k]-y[i,k], GRB.LESS_EQUAL, s)
        m.update()
        m.optimize()
        if m.status == GRB.OPTIMAL:
            print("Optimization successfull")
            print("Run time:",m.runtime)
            opt_val = -m.ObjVal
            print("Optimal value:",opt_val)
            if iter_input == 1:
                opt_val_list[ind] = opt_val
            else:
                opt_val_list[ind] = opt_val_list[ind]*iter_input/(iter_input+1) + opt_val/(iter_input+1)

In [ ]:
opt_val_list

This is the opt_val_list for standard PA attachment graph with n = 50, m = 3

opt_val_list = np.array([ 1.        ,  0.99999999,  0.9995002 ,  0.99402816,  0.97941715,
        0.95655252,  0.92859711,  0.89798283,  0.86589987, 0.84945249,  0.83785209])

In [ ]:
# Assigning saved values of opt_val_list
# Comment the following line if we calculate opt_val_list using the above code snippet.
opt_val_list = np.array([ 1.,  0.99999999,  0.9995002 ,  0.99402816,  0.97941715, 0.95655252,  0.92859711,  0.89798283,  0.86589987, 0.84945249,  0.83785209])

In [ ]:
# Careful about the path of the files!
FIGSIZE_REQD = 0.6
eps_list = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.985]
plt.style.use('ggplot')

# Plot averaged optimal precision curve
fig, ax = plt.subplots(figsize = figsize(FIGSIZE_REQD))
ax.plot(eps_list,opt_val_list, linewidth=3, label = 'Optimal precision')

# Plot Peeling algorithm precision and density
str = "../build/temp_model/theta_delta_peel.txt"
data_t = open(str,'r')
data_t = data_t.readlines()
theta_peel = []
delta_peel = []
for lin in data_t:
    temp_l=lin.split()
    theta_peel.append(float(temp_l[0]))
    delta_peel.append(float(temp_l[1]))
theta_avg = np.mean(theta_peel)
delta_avg = np.mean(delta_peel)
ax.plot(delta_avg, theta_avg, marker='o', markersize=5,\
        color = '#00008B', alpha = 1)
plt.scatter(delta_peel,theta_peel, alpha=0.5, label = r'\textsc{Peeling}',color = '#377eb8',edgecolors = 'white')
ax.set_xlabel(r'$\varepsilon$')
ax.set_ylabel(r'$\theta$')

# Plot Peeling+ algorithm precision and density
str_1 = "../build/temp_model/theta_delta_peel+.txt"
data_t = open(str_1,'r')
data_t = data_t.readlines()
theta_guess = []
delta_guess = []
for lin in data_t:
    temp_l=lin.split()
    theta_guess.append(float(temp_l[0]))
    delta_guess.append(float(temp_l[1]))
theta_guess_avg = np.mean(theta_guess)
delta_guess_avg = np.mean(delta_guess)
ax.plot(delta_guess_avg, theta_guess_avg, marker='o', markersize=5,color = '#006400', alpha = 1)
plt.scatter(delta_guess,theta_guess, alpha=0.5,color = '#4daf4a',\
            label = r'\textsc{Peeling}+',edgecolors = 'white')

# Plot Maximum density precision-1 algorithm precision and density
str_1 = "../build/temp_model/theta_delta_perf.txt"
data_t = open(str_1,'r')
data_t = data_t.readlines()
theta_guess = []
delta_guess = []
for lin in data_t:
    temp_l=lin.split()
    theta_guess.append(float(temp_l[0]))
    delta_guess.append(float(temp_l[1]))
theta_guess_avg = np.mean(theta_guess)
delta_guess_avg = np.mean(delta_guess)
ax.plot(delta_guess_avg, theta_guess_avg, marker='o', markersize=5,color = '#7a3d00', alpha = 1)
plt.scatter(delta_guess,theta_guess, alpha=0.5,color = '#ffff33',label = r'Perfect-precision', edgecolors = 'black')

ax.yaxis.set_ticks(np.arange(0.2, 1.2, 0.1))
ax.legend(loc=[0.1,0.1])
plt.savefig('../build/optimal_precision_LP.pdf',bbox_inches='tight',pad_inches = 0.05)

# Plot Recall Matrix

In [ ]:
width_box = 10
corr_matrix = np.zeros((width_box,width_box))
str = "../build/temp_model/Recall_matrix.txt"
data_t = open(str,'r')
data_t = data_t.readlines()
temp_l=data_t[2].split()
n = int(temp_l[0]) 
for lin in data_t[0:]:
    temp_l=lin.split()
    corr_matrix[int(temp_l[0]), int(temp_l[1])] = float(temp_l[2])

FIGSIZE_REQD = 0.6
plt.style.use('default')
fig, ax = plt.subplots(figsize = figsize(FIGSIZE_REQD))
cax = ax.matshow(corr_matrix, interpolation='nearest',cmap='viridis_r')
fig.colorbar(cax)
plt.savefig('../build/recall_matrix.pdf',bbox_inches='tight',pad_inches = 0.05)